## Connecting to the Alpha Vantage API to pull data

In [1]:
import requests
import pandas as pd
import numpy as np
from google.colab import userdata

# Retrieve the Alpha Vantage API key from Colab's secure storage
api_key = userdata.get('ALPHA_VANTAGE_API_KEY') ## Replace this with your own API Key

def get_weekly_data(symbol):
    """
    Retrieve weekly time series for a given stock symbol.
    """
    base_url = "https://www.alphavantage.co/query"
    function = "TIME_SERIES_WEEKLY"

    params = {
        "function": function,
        "symbol": symbol,
        "apikey": api_key
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()

        if "Error Message" in data:
            print(f"API Error: {data['Error Message']}")
            return None

        if "Note" in data:
            print(f"API Note: {data['Note']}")
            return None

        if "Weekly Time Series" not in data:
            print("Error: 'Weekly Time Series' not found in the API response")
            print("API Response:", data)
            return None

        df = pd.DataFrame.from_dict(data["Weekly Time Series"], orient="index")
        df.index = pd.to_datetime(df.index)
        df = df.astype(float)

        # Rename columns for clarity
        df.columns = ['open', 'high', 'low', 'close', 'volume']

        return df

    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return None
    except ValueError as e:
        print(f"JSON Decoding Error: {e}")
        return None
    except Exception as e:
        print(f"Unexpected Error: {e}")
        return None

def enrich_stock_data(df):
    """
    Add additional metrics and ratios to the stock DataFrame.
    """
    # 1. Market traded volume in USD
    df['volume_usd'] = df['close'] * df['volume']

    # 2. Weekly returns
    df['weekly_return'] = df['close'].pct_change()

    # 3. 4-week rolling average of closing price
    df['ma_4w'] = df['close'].rolling(window=4).mean()

    # 4. 8-week rolling average of closing price (changed from 12-week)
    df['ma_8w'] = df['close'].rolling(window=8).mean()

    # 5. Bollinger Bands (10-week, 2 standard deviations)
    df['bb_middle'] = df['close'].rolling(window=10).mean()
    bb_std = df['close'].rolling(window=10).std()
    df['bb_upper'] = df['bb_middle'] + (bb_std * 2)
    df['bb_lower'] = df['bb_middle'] - (bb_std * 2)

    # 6. Relative Strength Index (RSI) - 10-week
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=10).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=10).mean()
    rs = gain / loss
    df['rsi_10w'] = 100 - (100 / (1 + rs))

    # 7. Weekly volatility (standard deviation of returns)
    df['volatility'] = df['weekly_return'].rolling(window=8).std() * np.sqrt(52)

    return df

def compare_stocks(symbols, start_date=None, end_date=None):
    """
    Compare weekly data for multiple stock symbols and enrich with additional metrics.
    """
    results = {}
    for symbol in symbols:
        df = get_weekly_data(symbol)
        if df is not None:
            df = enrich_stock_data(df)
            if start_date:
                df = df[df.index >= start_date]
            if end_date:
                df = df[df.index <= end_date]
            results[symbol] = df
        else:
            print(f"Failed to retrieve data for {symbol}")

    return results

def display_recent_data(df, rows=5):
    """
    Display the most recent rows of data, excluding rows with NaN values.
    """
    # Reverse the DataFrame to start from the most recent data
    df_reversed = df.iloc[::-1]

    # Find the first row without NaN values
    first_full_row = df_reversed.dropna().index[0]

    # Display the specified number of rows from the first full row
    return df_reversed.loc[first_full_row:].head(rows)

# Example usage
symbols = ["AAPL", "MSFT", "GOOGL"]  # Apple, Microsoft, Google
start_date = "2023-01-01"
end_date = "2024-01-01"

stock_data = compare_stocks(symbols, start_date, end_date)

for symbol, df in stock_data.items():
    print(f"\n{symbol}:")
    print(display_recent_data(df))
    print(f"Shape of the DataFrame: {df.shape}")

# Print API key (first 5 characters) for debugging
if api_key:
    print(f"\nAPI Key (first 5 chars): {api_key[:5]}...")
else:
    print("\nAPI Key not found. Make sure you've added it to Colab's secure storage.")


AAPL:
               open    high     low   close       volume    volume_usd  \
2023-01-06  130.280  130.90  124.17  129.62  369948527.0  4.795273e+10   
2023-01-13  130.465  134.92  128.12  134.76  333335284.0  4.492026e+10   
2023-01-20  134.830  138.61  133.77  137.87  271823466.0  3.747630e+10   
2023-01-27  138.120  147.23  137.90  145.93  338655715.0  4.942003e+10   
2023-02-03  144.955  157.38  141.32  154.50  480249683.0  7.419858e+10   

            weekly_return     ma_4w      ma_8w  bb_middle    bb_upper  \
2023-01-06      -0.038142  137.0450  144.11875    145.248  161.946588   
2023-01-13      -0.022557  143.2650  146.79500    147.786  161.348187   
2023-01-20      -0.055232  147.3275  148.51250    150.335  162.528799   
2023-01-27      -0.055469  150.9975  150.65375    153.038  164.931052   
2023-02-03       0.023111  151.1925  152.44375    154.911  167.694957   

              bb_lower    rsi_10w  volatility  
2023-01-06  128.549412  24.154786    0.250127  
2023-01-13  1

Let's install Clickhouse Connect and pyopenssl

In [9]:
!pip install clickhouse-connect

!pip install pyopenssl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.9/977.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 44.3 MB/s eta 0:00:00


Define the functions to connect to and use Clickhouse

In [22]:
import clickhouse_connect
import pandas as pd
from datetime import datetime

def connect_to_clickhouse(host, port, user, password, database):
    """
    Establish a secure connection to ClickHouse using clickhouse-connect.
    """
    client = clickhouse_connect.get_client(
        host=host,
        port=port,
        username=user,
        password=password,
        database=database,
        secure=True
    )
    return client

def create_table_if_not_exists(client, table_name):
    """
    Create a table in ClickHouse if it doesn't already exist.
    """
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        date Date,
        symbol String,
        open Float64,
        high Float64,
        low Float64,
        close Float64,
        volume Int64,
        volume_usd Float64,
        weekly_return Float64,
        ma_4w Float64,
        ma_8w Float64,
        bb_middle Float64,
        bb_upper Float64,
        bb_lower Float64,
        rsi_10w Float64,
        volatility Float64
    ) ENGINE = MergeTree()
    ORDER BY (date, symbol)
    """
    client.command(create_table_query)

def insert_data_to_clickhouse(client, table_name, df):
    """
    Insert data from a pandas DataFrame into ClickHouse.
    """
    # Ensure the index (date) is included as a column
    df_reset = df.reset_index()

    # Rename the index column to 'date' if it's not already named that
    if 'date' not in df_reset.columns:
        df_reset = df_reset.rename(columns={df_reset.columns[0]: 'date'})

    # Convert date to datetime
    df_reset['date'] = pd.to_datetime(df_reset['date'])

    # Use the built-in DataFrame insertion method
    client.insert_df(table_name, df_reset)

def store_stock_data_in_clickhouse(stock_data, clickhouse_config):
    """
    Store stock data in ClickHouse.
    """
    client = connect_to_clickhouse(**clickhouse_config)

    for symbol, df in stock_data.items():
        table_name = f'stock_data_{symbol.lower()}'
        create_table_if_not_exists(client, table_name)

        # Ensure all required columns are present
        required_columns = ['open', 'high', 'low', 'close', 'volume', 'volume_usd', 'weekly_return', 'ma_4w', 'ma_8w', 'bb_middle', 'bb_upper', 'bb_lower', 'rsi_10w', 'volatility']
        missing_columns = [col for col in required_columns if col not in df.columns]

        if missing_columns:
            print(f"Warning: The following columns are missing for {symbol}: {missing_columns}")
            print(f"Available columns: {df.columns.tolist()}")
            continue

        df['symbol'] = symbol  # Add symbol column

        # Convert data types
        df['volume'] = df['volume'].astype('int64')
        for col in ['open', 'high', 'low', 'close', 'volume_usd', 'weekly_return', 'ma_4w', 'ma_8w', 'bb_middle', 'bb_upper', 'bb_lower', 'rsi_10w', 'volatility']:
            df[col] = df[col].astype('float64')

        try:
            insert_data_to_clickhouse(client, table_name, df)
            print(f"Data for {symbol} stored in ClickHouse table: {table_name}")
        except Exception as e:
            print(f"Error inserting data for {symbol}: {str(e)}")
            print(f"DataFrame shape: {df.shape}")
            print(f"DataFrame columns and types:")
            print(df.dtypes)
            print(f"First few rows of data:")
            print(df.head())
            print(f"Data being sent to ClickHouse:")
            print(df.reset_index().head())

            # Additional error information
            if hasattr(e, 'details'):
                print(f"Error details: {e.details}")


Now let's run it

In [23]:
clickhouse_config = {
    'host': 'INSERT YOUR HOST ADDRESS HERE',
    'port': 8443,  # Default ClickHouse port
    'user': 'INSERT YOUR USERNAME HERE',
    'password': 'INSERT YOUR PASSWORD HERE',
    'database': 'default'
}

store_stock_data_in_clickhouse(stock_data, clickhouse_config)

Data for AAPL stored in ClickHouse table: stock_data_aapl
Data for MSFT stored in ClickHouse table: stock_data_msft
Data for GOOGL stored in ClickHouse table: stock_data_googl


Now that we've stored the data successfully in our cloud database with Clickhouse, let's extract it and see if it worked


In [26]:
import clickhouse_connect
import pandas as pd

def extract_data_from_clickhouse(clickhouse_config, table_name, limit=None):
    """
    Extract data from a ClickHouse table and return it as a pandas DataFrame.

    :param clickhouse_config: Dictionary containing ClickHouse connection details
    :param table_name: Name of the table to query
    :param limit: Optional limit on the number of rows to return
    :return: pandas DataFrame containing the queried data
    """
    client = clickhouse_connect.get_client(
        host=clickhouse_config['host'],
        port=clickhouse_config['port'],
        username=clickhouse_config['user'],
        password=clickhouse_config['password'],
        database=clickhouse_config['database'],
        secure=True
    )

    query = f"SELECT * FROM {table_name}"
    if limit:
        query += f" LIMIT {limit}"

    result = client.query(query)

    # Extract column names and data
    column_names = [col[0] for col in result.column_names]
    data = result.result_rows

    # Create DataFrame
    df = pd.DataFrame(data, columns=column_names)

    return df


In [27]:
# Extract data for MSFT (limit to 10 rows as an example)
msft_data = extract_data_from_clickhouse(clickhouse_config, 'stock_data_msft', limit=10)
print("MSFT Data (first 10 rows):")
print(msft_data)
print("\n")

MSFT Data (first 10 rows):
            d     s        o       h       l       c          v             v  \
0  2023-01-06  MSFT  243.080  245.75  219.35  224.93  159562627  3.589042e+10   
1  2023-01-13  MSFT  226.450  239.90  226.41  239.23  131675747  3.150079e+10   
2  2023-01-20  MSFT  237.970  242.38  230.68  240.22  123872791  2.975672e+10   
3  2023-01-27  MSFT  241.100  249.83  230.90  248.16  198648466  4.929660e+10   
4  2023-02-03  MSFT  244.510  264.69  242.20  258.35  152686042  3.944644e+10   
5  2023-02-10  MSFT  257.440  276.76  254.78  263.10  196239002  5.163048e+10   
6  2023-02-17  MSFT  267.640  274.97  256.00  258.06  170244679  4.393334e+10   
7  2023-02-24  MSFT  254.480  256.84  248.10  249.22  105098500  2.619265e+10   
8  2023-03-03  MSFT  252.460  255.62  245.61  255.29  126840033  3.238099e+10   
9  2023-03-10  MSFT  256.425  260.12  247.60  248.59  117910562  2.931139e+10   

          w         m          m        b           b           b          r  \
0

In [28]:
# Extract data for AAPL
aapl_data = extract_data_from_clickhouse(clickhouse_config, 'stock_data_aapl')
print("AAPL Data:")
print(aapl_data.head())
print("\n")

AAPL Data:
            d     s        o       h       l       c          v             v  \
0  2023-01-06  AAPL  130.280  130.90  124.17  129.62  369948527  4.795273e+10   
1  2023-01-13  AAPL  130.465  134.92  128.12  134.76  333335284  4.492026e+10   
2  2023-01-20  AAPL  134.830  138.61  133.77  137.87  271823466  3.747630e+10   
3  2023-01-27  AAPL  138.120  147.23  137.90  145.93  338655715  4.942003e+10   
4  2023-02-03  AAPL  144.955  157.38  141.32  154.50  480249683  7.419858e+10   

          w         m          m        b           b           b          r  \
0 -0.038142  137.0450  144.11875  145.248  161.946588  128.549412  24.154786   
1 -0.022557  143.2650  146.79500  147.786  161.348187  134.223813  24.100793   
2 -0.055232  147.3275  148.51250  150.335  162.528799  138.141201  23.369458   
3 -0.055469  150.9975  150.65375  153.038  164.931052  141.144948  28.185418   
4  0.023111  151.1925  152.44375  154.911  167.694957  142.127043  34.660556   

          v  
0  0.25

In [29]:
# Extract data for GOOGL
googl_data = extract_data_from_clickhouse(clickhouse_config, 'stock_data_googl')
print("GOOGL Data:")
print(googl_data.tail())

GOOGL Data:
             d      s       o       h        l       c          v  \
47  2023-12-01  GOOGL  136.03  138.42  130.665  131.86  136564306   
48  2023-12-08  GOOGL  129.88  138.56  127.900  134.99  175979415   
49  2023-12-15  GOOGL  132.38  133.72  129.685  132.60  179845143   
50  2023-12-22  GOOGL  132.63  141.99  132.430  141.49  160821784   
51  2023-12-29  GOOGL  141.59  142.68  138.780  139.69   71126278   

               v         w         m           m         b           b  \
47  1.800737e+10 -0.023187  135.2350  138.173750  139.9955  152.793662   
48  2.375546e+10  0.018024  137.1925  140.714375  141.7095  154.253659   
49  2.384747e+10 -0.062831  137.3775  141.638125  142.2625  153.948312   
50  2.275467e+10  0.012886  139.8900  143.688125  143.3985  152.917338   
51  9.935630e+09  0.029176  141.1125  143.566875  142.9635  153.237584   

             b          r         v  
47  127.197338  33.845429  0.242285  
48  129.165341  45.265411  0.325073  
49  130.576688

Set up Vega-Altair package for graphing

In [30]:
!pip install altair vega_datasets

import altair as alt
import pandas as pd
import numpy as np

# Enable Altair to render in Colab
alt.renderers.enable('colab')

# If you want to save charts as PNG (optional)
!pip install altair_saver
!npm install -g vega-lite vega-cli canvas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.5/844.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
npm WARN EBADENGINE Unsupported engine {
npm WARN EBADENGINE   package: 'vega-lite@5.21.0',
npm WARN EBADENGINE   required: { node: '>=18' },
npm WARN EBADENGINE   current: { node: 'v16.20.2', npm: '8.19.4' }
npm WARN EBADENGINE }
npm WARN deprecated glob@7.2.3: Glob versions prior to v9 are no longer supported
npm WARN deprecated are-we-there-yet@2.0.0: This package is no longer supported.
npm WARN deprecated rimraf@3.0.2: Rimraf versions prior to v4 are no longer supported
npm WARN deprecated inflight@1.0.6: This module is not supported, and leaks memory. Do not use it. Check out lru-cache if you

Interactive Multi-Stock Price Comparison

In [34]:
# Combine data from all stocks into a single DataFrame
combined_df = pd.concat([
    df.reset_index().rename(columns={'index': 'date'}).assign(symbol=symbol)
    for symbol, df in stock_data.items()
])

# Ensure the date column is properly formatted
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Create the chart
chart = alt.Chart(combined_df).mark_line().encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('close:Q', title='Closing Price'),
    color='symbol:N',
    tooltip=['symbol', 'date', 'close']
).properties(
    width=800,
    height=400,
    title='Stock Price Comparison'
).interactive()

chart

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

Stock Volatility Comparison

In [35]:
# Combine data from all stocks into a single DataFrame
combined_df = pd.concat([
    df.reset_index().rename(columns={'index': 'date'}).assign(symbol=symbol)
    for symbol, df in stock_data.items()
])

# Ensure the date column is properly formatted
combined_df['date'] = pd.to_datetime(combined_df['date'])

chart = alt.Chart(combined_df).mark_line().encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('volatility:Q', title='Volatility'),
    color='symbol:N',
    tooltip=['symbol', 'date', 'volatility']
).properties(
    width=800,
    height=400,
    title='Stock Volatility Comparison'
).interactive()

chart

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

Volume Analysis

In [36]:
# Combine data from all stocks into a single DataFrame
combined_df = pd.concat([
    df.reset_index().rename(columns={'index': 'date'}).assign(symbol=symbol)
    for symbol, df in stock_data.items()
])

# Ensure the date column is properly formatted
combined_df['date'] = pd.to_datetime(combined_df['date'])

base = alt.Chart(combined_df).encode(x='date:T')

volume_chart = base.mark_bar(opacity=0.5).encode(
    y=alt.Y('volume:Q', title='Trading Volume'),
    color='symbol:N'
)

price_chart = base.mark_line().encode(
    y=alt.Y('close:Q', title='Closing Price'),
    color='symbol:N'
)

chart = alt.layer(volume_chart, price_chart).resolve_scale(
    y='independent'
).properties(
    width=800,
    height=400,
    title='Stock Price and Trading Volume'
).interactive()

chart

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

Technical Analysis

In [37]:
# Assuming we're using AAPL data for this example
df = stock_data['AAPL'].reset_index().rename(columns={'index': 'date'})
df['date'] = pd.to_datetime(df['date'])

base = alt.Chart(df).encode(x='date:T')

price_chart = base.mark_line(color='blue').encode(
    y=alt.Y('close:Q', title='Price')
)

ma_4w = base.mark_line(color='red').encode(
    y='ma_4w:Q'
)

ma_8w = base.mark_line(color='green').encode(
    y='ma_8w:Q'
)

bb_upper = base.mark_line(color='gray', strokeDash=[5,5]).encode(
    y='bb_upper:Q'
)

bb_lower = base.mark_line(color='gray', strokeDash=[5,5]).encode(
    y='bb_lower:Q'
)

chart = (price_chart + ma_4w + ma_8w + bb_upper + bb_lower).properties(
    width=800,
    height=400,
    title='AAPL Technical Analysis'
).interactive()

chart

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

Correlation Heatmap

In [38]:
# Combine weekly returns for all stocks
returns_df = pd.DataFrame({symbol: df['weekly_return'] for symbol, df in stock_data.items()})

# Calculate correlation matrix
corr_matrix = returns_df.corr()

# Melt the correlation matrix for use with Altair
melted_corr = corr_matrix.reset_index().melt('index', var_name='variable', value_name='correlation')

# Create heatmap
heatmap = alt.Chart(melted_corr).mark_rect().encode(
    x='index:O',
    y='variable:O',
    color=alt.Color('correlation:Q', scale=alt.Scale(scheme='blueorange'))
)

# Add text labels
text = heatmap.mark_text().encode(
    text=alt.Text('correlation:Q', format='.2f'),
    color=alt.condition(
        alt.datum.correlation > 0.5,
        alt.value('white'),
        alt.value('black')
    )
)

# Combine heatmap and text layers
chart = (heatmap + text).properties(
    width=400,
    height=300,
    title='Correlation of Weekly Returns'
).interactive()

chart

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)